In [1]:
import json
import numpy as np
import os
import pandas as pd
import re
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Union, Tuple


load_dotenv()
INDIVIDUAL_RUNS_PATH = os.getenv("INDIVIDUAL_RUNS_PATH")

In [2]:
# load csv
df = pd.read_csv(INDIVIDUAL_RUNS_PATH, sep=";") 
df.tail(2)

,query,final_answer,difficulty,task_id,query_num_listings,query_num_websites,num_webpage_visits,model_calls,input_tokens,output_tokens,...,multi_agent,timestamp,error_types,full_runtime,mean_time_difference,median_time_difference,shortest_time_difference,longest_time_difference,number_of_timestamps,start_url
21,Search for studio apartments in Liverpool City...,NaN,easy,0,NaN,NaN,NaN,113,NaN,NaN,...,1,20241115230509,"{""Failed to load resource: the server responde...",NaN,NaN,NaN,NaN,NaN,NaN,https://www.rightmove.co.uk/
22,Search for furnished studio and 1-bedroom apar...,NaN,easy,1,NaN,NaN,20.0,56,302087.0,2421.0,...,0,20241219014506,NaN,571.0,9.844828,10.0,0.0,16.0,59.0,https://www.google.com/


In [3]:

# Extract timestamps from a JSONL file and calculate metrics
def process_jsonl_file(file_path: str) -> Union[dict[str, Union[float, np.float64, int]], None]:
    # Initialize a list to store timestamps
    timestamps: List[datetime] = []

    # Open the JSONL file and process line by line
    with open(file_path, 'r') as file:
        for line in file:
            try:
                json_line = json.loads(line.strip())  # Parse the JSON line
                if "timestamp" in json_line:  # Extract the timestamp if it exists
                    timestamps.append(datetime.fromisoformat(json_line["timestamp"]))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {e}")

    # If less than two timestamps, we cannot compute meaningful metrics
    if len(timestamps) < 2:
        return None  # Return None to indicate insufficient data

    # Calculate metrics
    time_differences = [(timestamps[i + 1] - timestamps[i]).total_seconds() for i in range(len(timestamps) - 1)]
    metrics = {
        "full_runtime": (timestamps[-1] - timestamps[0]).total_seconds(),
        "mean_time_difference": np.mean(time_differences),
        "median_time_difference": np.median(time_differences),
        "shortest_time_difference": np.min(time_differences),
        "longest_time_difference": np.max(time_differences),
        "number_of_timestamps": len(timestamps),
    }

    return metrics


In [32]:
# Base directory for logs
logs_dir = "../logs"

# Loop through folders and update the DataFrame
for root, dirs, files in os.walk(logs_dir):
    if "archive" in root:
        continue

    for file in files:
        if file == "log.jsonl":
            # Extract the folder name from the path
            actual_folder_name = os.path.basename(root)  # Extract the relevant part
            file_path = os.path.join(root, file)  # Construct file path
            print(file_path)

            # Process the log.jsonl file
            metrics = process_jsonl_file(file_path)

            if metrics is None:
                continue  # Skip folders with insufficient data

            # Normalize the `folder_name` in DataFrame for matching
            df["normalized_folder_name"] = df["folder_name"].apply(lambda x: os.path.basename(x))

            # Add metrics to the DataFrame
            for key, value in metrics.items():
                df.loc[df["normalized_folder_name"] == actual_folder_name, key] = value


# Drop the temporary column used for matching
df.drop(columns=["normalized_folder_name"], inplace=True)




../logs/vision_magnetic_easy_01_20241215230819/log.jsonl
../logs/vision_magnetic_advanced_02_20241217204927/log.jsonl
../logs/vision_magnetic_advanced_01_20241217203222/log.jsonl
../logs/vision_magnetic_easy_00_20241215221928/log.jsonl
../logs/vision_magnetic_advanced_00_20241215233606/log.jsonl


In [27]:
# show unique values in column "model_name"
df["model_name"].unique()

array(['web_voyager', 'magentic', 'skyvern'], dtype=object)

In [33]:
# show rows with "magentic" as "model_name"
filtered_df = df[df['model_name'] == 'magentic']
filtered_df.head()

,query,final_answer,difficulty,id_query,query_num_listings,query_num_websites,num_webpage_visits,model_calls,input_tokens,output_tokens,...,vision_model,multi_agent,timestamp,error_types,full_runtime,mean_time_difference,median_time_difference,smallest_time_difference,longest_time_difference,number_of_timestamps
11,Look for studio apartments in Liverpool City C...,NaN,easy,0,NaN,NaN,9.0,78,759520.0,13687.0,...,1,1,20241215221928,NaN,1546.890970,6.090122,0.067536,0.000099,69.511999,255.0
12,Search for furnished studio and 1-bedroom apar...,NaN,easy,1,NaN,NaN,11.0,58,331968.0,10018.0,...,1,1,20241215230819,NaN,1160.140999,5.859298,0.024781,0.000088,206.176920,199.0
13,Find pet-friendly studio flats for rent London...,NaN,advanced,0,NaN,NaN,6.0,34,393628.0,4886.0,...,1,1,20241215233606,NaN,853.379934,7.757999,0.123054,0.000091,83.561034,111.0
14,Find new-built detached houses for rent in Gre...,NaN,advanced,1,NaN,NaN,4.0,28,342064.0,5932.0,...,1,1,20241217203222,NaN,846.396770,9.101041,0.031927,0.000093,95.811207,94.0
15,Find furnished student apartments near the Uni...,NaN,advanced,2,NaN,NaN,6.0,37,149088.0,5909.0,...,1,1,20241217204927,NaN,354.474083,3.029693,0.021450,0.000101,17.960466,118.0


In [34]:
# save csv
df.to_csv(INDIVIDUAL_RUNS_PATH, sep=";", index=False)